### Installing Dependencies

In [6]:
%pip install langchain
%pip install langchain-ollama
%pip install langchain_community
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.5.0-cp313-cp313-win_amd64.whl.metadata (14 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ------------------------ --------------- 1.6/2.5 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 5.4 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ----- --------------------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Load ENV file

In [7]:
from dotenv import load_dotenv
import os
load = load_dotenv('./../.env')

print(os.getenv("LANGSMITH_API_KEY"))

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


#### Create LLM object

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    base_url="http://localhost:11434",
    model = "qwen2.5:3b",
    temperature=0.5,
    max_tokens = 250
)
# response = llm.invoke("hello, How are you doing?")

# print(response)

content="Hello! I'm an artificial intelligence, so I don't have feelings or experiences like humans do, but I'm here and ready to assist you. How can I help you today?" additional_kwargs={} response_metadata={'model': 'qwen2.5:3b', 'created_at': '2025-04-13T01:19:51.6642121Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6177837600, 'load_duration': 5598021300, 'prompt_eval_count': 36, 'prompt_eval_duration': 313039300, 'eval_count': 38, 'eval_duration': 265775200, 'message': Message(role='assistant', content='', images=None, tool_calls=None), 'model_name': 'qwen2.5:3b'} id='run-3950707f-cdcd-4261-bfad-1e6cd38001fa-0' usage_metadata={'input_tokens': 36, 'output_tokens': 38, 'total_tokens': 74}


### Message history with ChatMessageHistory

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory


In [9]:
#ChatMessage History

template = ChatPromptTemplate.from_messages([
     ('placeholder', "{history}"),
    ('human', "{prompt}")
])

chain = template | llm | StrOutputParser()

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="prompt",
    history_messages_key="history")

session_id = "kylie"

get_session_history(session_id).clear()

response1 = history.invoke({"prompt": "What is the distance between earth and sun?"},
                          config = {"configurable": {"session_id": session_id}})

response2 = history.invoke({"prompt": "How about from moon"},
                           config = {"configurable": {"session_id": session_id}})

print(response1)
print("\n\n")
print(response2)



The average distance from Earth to the Sun is approximately 93 million miles (or about 150 million kilometers). This distance is commonly referred to as one astronomical unit (AU). However, it's important to note that this value can vary slightly due to the elliptical shape of Earth's orbit.



The average distance between the Earth and the Moon is approximately 385,000 kilometers (or about 239,400 miles). This distance varies because the orbits of both bodies are not perfectly circular; they follow an elliptical path around each other. The closest point in their orbit (perigee) brings them about 363,104 kilometers apart, while the farthest point (apogee) is about 405,696 kilometers apart.


### ChatMessage History with SqlChatMessageHistory

In [10]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def get_session_history(session_id):
    return SQLChatMessageHistory(session_id=session_id, connection_string="sqlite:///chathistory.db")

template = ChatPromptTemplate.from_messages([
     ('placeholder', "{history}"),
    ('human', "{prompt}")
])

chain = template | llm | StrOutputParser()

history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="prompt",
    history_messages_key="history")


session_id = "kylie"

get_session_history(session_id).clear()

response1 = history.invoke({"prompt": "What is the distance between earth and sun?"},
                          config = {"configurable": {"session_id": session_id}})

response2 = history.invoke({"prompt": "How about from moon"},
                          config = {"configurable": {"session_id": session_id}})

print(response1)
print("\n\n")
print(response2)



C:\Users\noort\AppData\Local\Temp\ipykernel_3180\1273764868.py:23: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  get_session_history(session_id).clear()


The average distance between Earth and the Sun is approximately 93 million miles (or about 149.6 million kilometers). This distance is commonly referred to as one astronomical unit (AU). 

It's important to note that this value can vary slightly due to the elliptical orbit of Earth around the Sun, but for most calculations and everyday purposes, using 93 million miles or 149.6 million kilometers is accurate enough.



The average distance between the Moon and Earth is approximately 238,855 miles (or about 384,400 kilometers). This value can also vary slightly due to the elliptical orbit of the Moon around Earth.

For most calculations and everyday purposes, using this precise figure is accurate enough.
